In [ ]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

from topostats.grain_finding_haribo_unet import (
    predict_unet,
    load_model,
    predict_unet_multiclass_and_get_angle,
    mean_iou,
)

from topostats.haribonet_process_grain_bound import process_grain

In [ ]:
# Load model
model_path = "./haribonet_multiclass_improved_norm_big_95_good2024-01-14_16-06-47.h5"
print(f"Loading Unet model: {model_path}")
model = load_model(model_path=model_path, custom_objects={"mean_iou": mean_iou})
print(f"Loaded Unet model: {model_path}")

In [ ]:
# Get ring images
DATA_DIR = Path("/Users/sylvi/topo_data/hariborings/cas9_crops_all/ON_SC/")
IMAGE_SAVE_DIR = DATA_DIR / "output_plots"
IMAGE_SAVE_DIR.mkdir(exist_ok=True)

image_files = sorted(DATA_DIR.glob("*.npy"))
print(f"num images: {len(image_files)}")

In [ ]:
angles = []

for file in image_files:
    image = np.load(file)

    try:
        angle, predicted_ring_mask = process_grain(
            cropped_image=image,
            model=model,
            grain_number=0,
            IMAGE_SAVE_DIR=IMAGE_SAVE_DIR,
            filename=file.stem,
        )

        angles.append(angle)
    except Exception as e:
        print(f"Failed on {file}, {e}")

In [ ]:
# Plot angles as a kde

import seaborn as sns

sns.distplot(angles, bins=100)
plt.xlabel("Angle (degrees)")
plt.ylabel("Density")
plt.title("Angle distribution for Haribo rings")
plt.savefig(IMAGE_SAVE_DIR / "angle_distribution.png")
plt.show()

# Save the angles to a file
np.save(IMAGE_SAVE_DIR / "angles.npy", angles)